In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from rlrom.envs import *

2025-03-27 09:29:38.445186: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-27 09:29:38.460497: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-27 09:29:38.465012: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-27 09:29:38.476077: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-27 09:29:39.206675: W tensorflow/compiler/tf2

In [2]:
pendul_cfg = cfg_envs['Pendulum']

## Training

In [3]:
import gymnasium as gym
# Create the Pendulum environment
env = gym.make(pendul_cfg['env_name'])




In [ ]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import SAC
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

# --- Parameters ---
ENV_ID = "Pendulum-v1"
# Hyperparameters inspired by RL Baselines Zoo3 for SAC on Pendulum-v1
# (Note: Zoo often uses more advanced setups like VecNormalize, but this provides the core SAC params)
tuned_params = {
    'learning_rate': 7.3e-4, # Often represented as linear_schedule(7.3e-4) in the zoo, but constant works too
    'buffer_size': 50000,
    'learning_starts': 100,
    'batch_size': 256,
    'tau': 0.005,
    'gamma': 0.99,
    'train_freq': 1,
    'gradient_steps': 1,
    'ent_coef': 'auto', # Automatically tune entropy coefficient
    'target_update_interval': 1,
    'target_entropy': 'auto',
    'use_sde': False, # State Dependent Exploration - can sometimes help, but often not needed for Pendulum
    'sde_sample_freq': -1,
    'policy_kwargs': dict(net_arch=[64, 64]) # Neural network architecture
}
TOTAL_TIMESTEPS = 50_000 # Adjust as needed, 50k-100k is often enough for Pendulum
N_EVAL_EPISODES = 10
MODEL_SAVE_PATH = "./sac_pendulum_tuned"

# --- Environment Setup ---
# Create a single environment instance, wrapped with Monitor for logging
env = gym.make(ENV_ID)
env = Monitor(env)

# For evaluation later
eval_env = gym.make(ENV_ID)
eval_env = Monitor(eval_env) # Wrap eval env too if you want stats from evaluation


# --- Model Definition ---
# Use the tuned hyperparameters
model = SAC(
    "MlpPolicy",
    env,
    verbose=1, # Print training progress
    **tuned_params
)

print(f"--- Training SAC on {ENV_ID} for {TOTAL_TIMESTEPS} timesteps ---")
print(f"Hyperparameters: {tuned_params}")

# --- Training ---
# The learn() method handles the training loop
model.learn(total_timesteps=TOTAL_TIMESTEPS, log_interval=10) # Log every 10 training updates

print("--- Training Finished ---")

# --- Saving the Model ---
model.save(MODEL_SAVE_PATH)
print(f"Model saved to {MODEL_SAVE_PATH}.zip")

# --- Evaluation ---
print("--- Evaluating Trained Agent ---")
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=N_EVAL_EPISODES)
print(f"Evaluation over {N_EVAL_EPISODES} episodes:")
print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

# --- Optional: Load and Run Trained Agent ---
# del model # Remove trained model from memory
# loaded_model = SAC.load(MODEL_SAVE_PATH, env=eval_env)
# print("--- Running Loaded Agent ---")
# obs, _ = eval_env.reset()
# total_reward = 0
# terminated = False
# truncated = False
# while not terminated and not truncated:
#     action, _states = loaded_model.predict(obs, deterministic=True) # Use deterministic actions for evaluation
#     obs, reward, terminated, truncated, info = eval_env.step(action)
#     total_reward += reward
#     # Optional: render the environment
#     # eval_env.render()
# print(f"Reward in one episode run: {total_reward}")

eval_env.close()
env.close()

## Testing

In [3]:
import gymnasium as gym
from stable_baselines3 import PPO
import stlrom
import numpy as np
from rlrom.wrappers import STLWrapper

In [4]:
env= gym.make(cfg_envs['Pendulum']['env_name'], render_mode= 'human')
loaded_model = PPO.load("ppo_pendulum_optimized", env=env)


/home/alex/python_venvs/rlrom_env/lib/python3.12/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [5]:
driver = stlrom.STLDriver()
driver.parse_string(pendul_cfg['specs'])
env = STLWrapper(env, driver, signals_map=pendul_cfg)

In [6]:
print(env.signals_map)

{'torque': 'action[0]', 'cos_theta': 'obs[0]', 'sin_theta': 'obs[1]', 'theta_dot': 'obs[2]', 'reward': 'reward'}


In [6]:
obs,_= env.reset()
terminated = False
truncated = False
total_reward =0


In [7]:
obs,_= env.reset()
terminated = False
truncated = False
total_reward =0
while not terminated and not truncated:
    action, _states = loaded_model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    total_reward += reward
    env.render()  # Optional: render the environment

print(f"Total reward during test: {total_reward}")
env.close()

Total reward during test: -135.08449865717606


In [8]:
env.close()